<a href="https://colab.research.google.com/github/Muskan-2725/keyword_spotting_tinyml_project/blob/main/Keyword_Spotting_TinyML_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "kws_project.zip" -d kws_project


Streaming output truncated to the last 5000 lines.
  inflating: kws_project/kws project /yes/1cb788bc_nohash_0.wav  
  inflating: kws_project/__MACOSX/kws project /yes/._1cb788bc_nohash_0.wav  
  inflating: kws_project/kws project /yes/365531e2_nohash_0.wav  
  inflating: kws_project/__MACOSX/kws project /yes/._365531e2_nohash_0.wav  
  inflating: kws_project/kws project /yes/b8874962_nohash_2.wav  
  inflating: kws_project/__MACOSX/kws project /yes/._b8874962_nohash_2.wav  
  inflating: kws_project/kws project /yes/a04817c2_nohash_1.wav  
  inflating: kws_project/__MACOSX/kws project /yes/._a04817c2_nohash_1.wav  
  inflating: kws_project/kws project /yes/0a196374_nohash_3.wav  
  inflating: kws_project/__MACOSX/kws project /yes/._0a196374_nohash_3.wav  
  inflating: kws_project/kws project /yes/4bc2c2c5_nohash_2.wav  
  inflating: kws_project/__MACOSX/kws project /yes/._4bc2c2c5_nohash_2.wav  
  inflating: kws_project/kws project /yes/60472d26_nohash_0.wav  
  inflating: kws_project/

In [ ]:
!ls kws_project

'kws project '	 __MACOSX


In [ ]:
!pip install librosa tensorflow numpy

import libraries

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf


loading sample audio

In [18]:
audio, sr = librosa.load("/content/kws_project/kws_project/yes/004ae714_nohash_0.wav", sr=16000)

print("Sample rate:", sr)
print("Audio length:", len(audio))


Sample rate: 16000
Audio length: 16000


mfcc feature extraction

In [19]:
SAMPLES = 16000
MFCC_COUNT = 13

def extract_mfcc(file_path):
    signal, sr = librosa.load(file_path, sr=16000)

    if len(signal) < SAMPLES:
        signal = np.pad(signal, (0, SAMPLES - len(signal)))
    else:
        signal = signal[:SAMPLES]

    mfcc = librosa.feature.mfcc(
        y=signal,
        sr=sr,
        n_mfcc=MFCC_COUNT
    )

    return mfcc.T


In [20]:
X, y = [], []
DATASET_PATH = "kws_project/kws_project"

labels = os.listdir(DATASET_PATH)
labels = [l for l in labels if os.path.isdir(os.path.join(DATASET_PATH, l))]

for label in labels:
    folder = os.path.join(DATASET_PATH, label)
    for file in os.listdir(folder):
        if file.endswith(".wav"):
            mfcc = extract_mfcc(os.path.join(folder, file))
            X.append(mfcc)
            y.append(label)

X = np.array(X)

print("MFCC shape:", X.shape)
print("Total samples:", len(X))
print("Classes:", set(y))


MFCC shape: (11952, 32, 13)
Total samples: 11952
Classes: {'unknown', 'no', '_background_noise_', 'yes', 'stop'}


label encoding

In [21]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

X = X[..., np.newaxis]

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_encoded = to_categorical(y_encoded)

print("Final X shape:", X.shape)
print("Final y shape:", y_encoded.shape)
print("Classes:", encoder.classes_)


Final X shape: (11952, 32, 13, 1)
Final y shape: (11952, 5)
Classes: ['_background_noise_' 'no' 'stop' 'unknown' 'yes']


cnn model

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(8, (3,3), activation='relu', input_shape=X.shape[1:]),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(len(encoder.classes_), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 11, 8)      │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 5, 8)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 600)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        19,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,477 (76.08 KB)

 Trainable params: 19,477 (76.08 KB)

 Non-trainable params: 0 (0.00 B)

training

In [23]:
history = model.fit(
    X, y_encoded,
    epochs=5,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.6857 - loss: 6.3021 - val_accuracy: 0.7169 - val_loss: 4.2392
Epoch 2/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8670 - loss: 0.9212 - val_accuracy: 0.6228 - val_loss: 4.7982
Epoch 3/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8941 - loss: 0.5965 - val_accuracy: 0.8536 - val_loss: 2.2016
Epoch 4/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9077 - loss: 0.4313 - val_accuracy: 0.7691 - val_loss: 2.7573
Epoch 5/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9222 - loss: 0.3581 - val_accuracy: 0.8607 - val_loss: 1.9112


tflite conversion

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("kws_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model saved as kws_model.tflite")


Saved artifact at '/tmp/tmp2wwdj2pg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 13, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  140043612907920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140043607549776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140043612909456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140043612899472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140043612905040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140043612903120: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ TFLite model saved as kws_model.tflite


In [25]:
interpreter = tf.lite.Interpreter(model_path="kws_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])


Input shape: [ 1 32 13  1]
Output shape: [1 5]


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [32]:
from google.colab import files

uploaded = files.upload()


Saving 0a2b400e_nohash_2.wav to 0a2b400e_nohash_2.wav


In [39]:
!rm 0a2b400e_nohash_0.wav

uploading audio file for demo

In [40]:
!ls

0a2b400e_nohash_2.wav  kws_project	sample_data
kws_model.tflite       kws_project.zip


preprocess audio

In [34]:
import librosa
import numpy as np

SAMPLES = 16000
MFCC_COUNT = 13

def preprocess_audio(file_path):
    signal, sr = librosa.load(file_path, sr=16000)

    if len(signal) < SAMPLES:
        signal = np.pad(signal, (0, SAMPLES - len(signal)))
    else:
        signal = signal[:SAMPLES]

    mfcc = librosa.feature.mfcc(
        y=signal,
        sr=sr,
        n_mfcc=MFCC_COUNT
    )

    mfcc = mfcc.T
    mfcc = mfcc[np.newaxis, ..., np.newaxis]  # shape (1, 32, 13, 1)
    return mfcc


In [35]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="kws_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])


Input shape: [ 1 32 13  1]
Output shape: [1 5]


predict keyword

In [38]:
labels = ['_background_noise_', 'no', 'stop', 'unknown', 'yes']

input_data = preprocess_audio("0a2b400e_nohash_2.wav")

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

output = interpreter.get_tensor(output_details[0]['index'])
predicted_label = labels[np.argmax(output)]

print("🎯 Predicted Keyword:", predicted_label)


🎯 Predicted Keyword: no
